In [131]:
#Santiago Gonzalez de Cosio Romero
#A01640329

In [132]:
#model design
import agentpy as ap

#visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random


In [133]:
MOVES = [(1,0), (0,1), (-1,0), (0,-1)]

class ModeloCeldas(ap.Model):

    def setup(self):
        # Create agents (celdas sucias, limpia)
        self.limpia = ap.AgentList(self,10)
        
        n_basura = int(self.p['sucio_d'] * (self.p['m'] * self.p['n']))
        self.sucio = ap.AgentList(self,n_basura)
       

        # Create grid (cuarto) 
        self.habitaciones = ap.Grid(self, [self.p['m'],self.p['n']], track_empty=True)
        self.habitaciones.add_agents(self.sucio, random=True, empty=True) 
        self.habitaciones.add_agents(self.limpia,[(1,1)]*10)

        # Initiate a dynamic variable for all dirty cells
        # Condition 0: dirty, 1: cleaning, 2: cleaned
        self.limpia.type_agent = 0
        self.sucio.type_agent = 1


    def step(self):

        limpiadores = self.limpia
        # Select cleaning cells


        # Start cleaning
        for limpia in limpiadores:
            for neighbor in self.habitaciones.neighbors(limpia):
                if neighbor.type_agent == 1:
                    neighbor.type_agent = 2 # Neighbor starts being cleaned
                    break
            else:
                self.habitaciones.move_by(limpia, random.choice(MOVES))
        
        sucio_remaining = self.sucio.select(self.sucio.type_agent == 1)
        # Stop simulation if no dirty cell is left
        if len(sucio_remaining) == 0:
            self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        hab_limpias = len(self.sucio.select(self.sucio.type_agent == 2))
        self.report('Percentage of cleaned cells',
                    hab_limpias / len(self.sucio))


In [134]:
# Define parameters

parameters = {
    'sucio_d': 0.6, # Percentage of grid covered by habitaciones
    'n': 50,
    'm': 40, # Height and length of the grid
    'steps': 100,
}

In [135]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.habitaciones.attr_grid('type_agent')
    color_dict = {0:'#00FF00', 1:'#964B00', 2:'#89CFF0', None:'#d5e5d5'} 
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulacion de limpieza de habitaciones\n"
                 f"Paso-tiempo: {model.t}, Porcentaje de celdas limpias: {(len(model.sucio.select(model.sucio.type_agent == 2)) / len(model.sucio))*100}")

fig, ax = plt.subplots()
model = ModeloCeldas(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))